Nome: Fabio Grassiotto  
RA: 890441

# Aula 10_11 - Implementação do RAGAS

-Implementar o RAGAS com o LLaMA-3 70B para avaliar a qualidade das 50 anotações do IIRC usadas no exercício passado.  
-O RAGAS considera context, question, answer, keys que estão disponíveis no conjunto de teste do IIRC.  
Opcional:  
-Avaliar as respostas do exercício da aula 9_10  
-Usar multi agents  


## Setup Environment
### Install packages

In [1]:
%%capture
%pip install -q torch
%pip install groq
%pip install llama-index-llms-groq
%pip install langchain
%pip install langchain-groq
%pip install -U sentence-transformers
%pip install faiss-cpu
%pip install rank_bm25
%pip install bs4
%pip install spacy
!python -m spacy download en_core_web_sm
%pip install ragas

### Imports

In [2]:
import os
import sys
import json
import torch
from collections import Counter
import string
import re
from bs4 import BeautifulSoup
from tqdm import tqdm
import spacy
import pandas as pd
import warnings
warnings.simplefilter('ignore')

# Langchain
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain.tools import tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate

# Sentence Transformers
from sentence_transformers import SentenceTransformer, CrossEncoder
import faiss

# Ragas
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
from ragas import evaluate

### Collab setup

In [3]:
# Colab environment
IN_COLAB = 'google.colab' in sys.modules

if (IN_COLAB):
    # Google Drive
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    project_folder="/content/drive/MyDrive/Classes/IA024/Aula_9_10"
    os.chdir(project_folder)
    !ls -la

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


## Globals

In [4]:
NUM_QUESTIONS = 50
LLM_MODEL_NAME="llama3-70b-8192"
SEARCH_MODEL_NAME = "sentence-transformers/msmarco-MiniLM-L-6-v3"
CROSSENCODER_MODEL_NAME = "cross-encoder/ms-marco-MiniLM-L-6-v2"

## IIRC Dataset

In [5]:
if not os.path.exists("dataset/context_articles.json"):
    !wget http://jamesf-incomplete-qa.s3.amazonaws.com/context_articles.tar.gz
    !tar -xzf context_articles.tar.gz --directory dataset
    !rm context_articles.tar.gz

In [6]:
test_dataset  = json.load(open('dataset/test_questions.json', 'r'))
articles = json.load(open('dataset/context_articles.json', 'r'))

In [7]:
test_dataset[5]

{'answer': {'type': 'span',
  'answer_spans': [{'text': 'White Hart Lane',
    'passage': 'Chris Brunt',
    'type': 'answer',
    'start': 622,
    'end': 637}]},
 'question': 'Which stadium where Brunt played can hold more people, The Hawthorns or White Hart Lane?',
 'context': [{'text': 'at The Hawthorns',
   'passage': 'main',
   'indices': [139, 155]},
  {'text': 'at White Hart Lane', 'passage': 'main', 'indices': [619, 637]},
  {'text': 'The Hawthorns is an all-seater football stadium in West Bromwich, West Midlands, England, with a capacity of 26,688',
   'passage': 'The Hawthorns',
   'indices': [0, 115]},
  {'text': ' Its capacity varied over the years; when changed to all-seater it had a capacity of 36,284 before demolition.',
   'passage': 'White Hart Lane',
   'indices': [132, 242]}],
 'question_links': ['The Hawthorns', 'White Hart Lane'],
 'title': 'Chris Brunt'}

### Load and parse

In [8]:
# Grab the first 50 questions  # type: ignorewith an answer
def grab_documents():

  questions_found = []
  num_questions_found = 0
  documents = []
  all_titles = []

  for item in test_dataset:
    question = item['question']
    answer = item['answer']
    answer_type = answer['type']

    if answer_type == 'binary' or answer_type == 'value':
      final_answer = answer['answer_value']
    elif answer_type == 'span':
      final_answer = answer['answer_spans'][0]['text']
    elif answer_type == 'none':
      final_answer = 'none'
    else:
      final_answer = 'An error perhaps, bad type'
      print(answer_type)

    if (final_answer == 'none'):
      # Skip this one.
      continue
    else:
      # Thats a good question.
      
      # First add some extra info in the context-part
      for context in item['context']:
          # Cleanup html tags
          soup = BeautifulSoup(context['text'], 'html.parser')
          clean_text = soup.get_text()

          documents.append({
                  "title": item['title'].lower(),
                  "content": clean_text
            }
          )
          
      all_titles.append(item['title'].lower())

      # And then grab the text from the articles json 
      for link in item["question_links"]:
        if link.lower() in articles and link.lower() not in all_titles:
          # Cleanup html tags
          soup = BeautifulSoup(articles[link.lower()], 'html.parser')
          clean_text = soup.get_text()

          documents.append({
            "title": link,
            "content": clean_text
          })
        all_titles.append(link.lower())

      # Fix leading, trailing and excessive spaces between words.
      question = question.strip()
      question = " ".join(question.split())

      questions_found.append({"Id": num_questions_found, "Question": question.strip(), "Answer": final_answer.strip()})
      num_questions_found += 1
      if (num_questions_found == NUM_QUESTIONS):
        # found our questions
        break

  return questions_found, documents, all_titles

questions_to_ask, documents, all_titles = grab_documents()

### Sequence segmentation (3 per document)

In [9]:
contents_list = []
nlp = spacy.load('en_core_web_sm')

for t in tqdm(documents):
    paragraph = t.get('content')
    paragraph = paragraph[:512] 

    sentences = list(sent.text for sent in nlp(paragraph).sents)
    sentences = sentences[:3]
    excerpt = ""
    for s in sentences:
        excerpt += s
    contents_list.append(excerpt)

100%|██████████| 192/192 [00:01<00:00, 124.90it/s]


### Inspecting data

In [10]:
len(questions_to_ask)

50

In [11]:
len(contents_list)

192

In [12]:
contents_list[:10]

['he Palici the sons of Zeus',
 'in Greek mythology',
 'Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion',
 "Greek mythology is the body of myths originally told by the ancient Greeks and a genre of Ancient Greek folklore.These stories concern the origin and the nature of the world, the lives and activities of deities, heroes, and mythological creatures, and the origins and significance of the ancient Greeks' own cult and ritual practices.Modern scholars study the myths in an attempt to shed light on the religious and political institutions of ancient Greece and its civilization, and to gain understanding of the",
 'Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion, who rules as king of the gods of Mount Olympus.His name is cognate with the first element of his Roman equivalent Jupiter.His mythologies and powers are similar, though not identical, to those of Indo-E

In [13]:
all_titles[:10]

['palici',
 'greek mythology',
 'zeus',
 'giovanni messe',
 'world war i',
 'giovanni messe',
 'world war i',
 'casino royale (1967 film)',
 'angela scoular',
 "on her majesty's secret service (film)"]

In [14]:
questions_to_ask[0]

{'Id': 0,
 'Question': 'What is Zeus know for in Greek mythology?',
 'Answer': 'sky and thunder god'}

## FAISS Indexing

In [15]:
model = SentenceTransformer(SEARCH_MODEL_NAME)
model.to(device)

embeddings = model.encode(contents_list, show_progress_bar=True)

Batches: 100%|██████████| 6/6 [00:00<00:00,  9.17it/s]


In [18]:
# Create a FAISS index from the embeddings
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

## Langchain Groq API

In [19]:
def load_groq_key():
    try:
        # Open and read the entire content of the file
        with open("groq-key.txt", 'r') as file:
            contents = file.read()
        
        return contents
    
    except FileNotFoundError:
        print(f"The file does not exist.")
        return None
    except Exception as e:
        # Handle other potential exceptions (e.g., permission errors)
        print(f"An error occurred while reading the file: {str(e)}")
        return None
    
chat = ChatGroq(temperature=0, groq_api_key=load_groq_key(), model_name=LLM_MODEL_NAME)

In [20]:
system = "You are a helpful assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
chain.invoke({"text": "Explain the importance of low latency LLMs."})

AIMessage(content='Low-latency Large Language Models (LLMs) are crucial in various applications where real-time or near-real-time processing is essential. Here are some reasons why low-latency LLMs are important:\n\n1. **Interactive Systems**: In applications like chatbots, virtual assistants, and conversational AI, low-latency LLMs enable rapid response times, creating a more natural and engaging user experience. This is particularly important in customer-facing applications where delayed responses can lead to frustration and abandonment.\n2. **Real-time Decision Making**: In areas like finance, healthcare, and cybersecurity, timely decision-making is critical. Low-latency LLMs can quickly analyze vast amounts of data, enabling swift identification of patterns, anomalies, and insights, which inform timely decisions.\n3. **Live Content Generation**: Low-latency LLMs are essential for generating content in real-time, such as live subtitles, closed captions, or automated summarization of

## ReAct Agent Implementation

### Prompt from Langchain (https://smith.langchain.com/hub/hwchase17/react-chat)

This prompt was modified:

- to remove the chat history, so every question is a new chat with the LLM.
- to request the model to use short answers, and to answer with numerals instead of words for numbers, to increase our F1 score.

In [21]:
LANGCHAIN_CHAT_SYSTEM_HEADER = """\

Assistant is a large language model trained by Meta AI.
Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.
Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.
Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

TOOLS:
------
Assistant has access to the following tools:
{tools}
To use a tool, please use the following format:
```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```
When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:
```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```
Your final answer must be short, no more than 10 words, and use numerals instead of words for numbers.
Begin!
New input: {input}
{agent_scratchpad}
"""

### Agent tool definition

Top 5 hits returned from the search, using a crossencoder for reranking.

In [27]:
# Prepare context list for Ragas evaluation
contexts = []
for item in tqdm(questions_to_ask):
    question_id = item.get("Id")
    question = item.get('Question')
                        
    xq = model.encode([question])
    D, I = index.search(xq, 5)  # search
    results = []
    for i in range(0, 5):
        results.append(contents_list[I[0][i]]) 

    crossencoder = CrossEncoder(CROSSENCODER_MODEL_NAME)
    # Concatenate the query and all passages and predict the scores for the pairs [query, passage]
    crossencoder_inputs = [[question, passage] for passage in results]
    scores = crossencoder.predict(crossencoder_inputs)

    # Sort the scores in decreasing order
    reranked_results = [{"input": inp, "score": score} for inp, score in zip(crossencoder_inputs, scores)]
    reranked_results = sorted(reranked_results, key=lambda x: x["score"], reverse=True)

    cross_enc_res = []
    for res in reranked_results:
        cross_enc_res.append(res['input'][1])

    contexts.append(cross_enc_res)
    
@tool
def get_top5_rerank(question: str) -> list[str]:
    """Returns a list of five strings with texts to help solve a question, using a cross-encoder for re-ranking."""
    xq = model.encode([question])
    D, I = index.search(xq, 5)  # search
    results = []
    for i in range(0, 5):
        results.append(contents_list[I[0][i]]) 

    crossencoder = CrossEncoder(CROSSENCODER_MODEL_NAME)
    # Concatenate the query and all passages and predict the scores for the pairs [query, passage]
    crossencoder_inputs = [[question, passage] for passage in results]
    scores = crossencoder.predict(crossencoder_inputs)

    # Sort the scores in decreasing order
    reranked_results = [{"input": inp, "score": score} for inp, score in zip(crossencoder_inputs, scores)]
    reranked_results = sorted(reranked_results, key=lambda x: x["score"], reverse=True)

    cross_enc_res = []
    for res in reranked_results:
        cross_enc_res.append(res['input'][1])
    return cross_enc_res

100%|██████████| 50/50 [00:38<00:00,  1.30it/s]


## Execution Run

### Evaluation functions

In [29]:
def normalize_answer(s):
  """
  Taken from the official evaluation script for v1.1 of the SQuAD dataset.
  Lower text and remove punctuation, articles and extra whitespace.
  """

  def remove_articles(text):
      return re.sub(r"\b(a|an|the)\b", " ", text)

  def white_space_fix(text):
      return " ".join(text.split())

  def remove_punc(text):
      exclude = set(string.punctuation)
      return "".join(ch for ch in text if ch not in exclude)

  def lower(text):
      return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))

def token_f1_score(prediction, ground_truth):
  """
  Taken from the official evaluation script for v1.1 of the SQuAD dataset.
  """
  prediction_tokens = normalize_answer(prediction).split()
  ground_truth_tokens = normalize_answer(ground_truth).split()
  common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
  num_same = sum(common.values())
  if num_same == 0:
      return 0
  precision = 1.0 * num_same / len(prediction_tokens)
  recall = 1.0 * num_same / len(ground_truth_tokens)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

### Analysing the output for 50 questions

In [30]:
def evaluate(executor):
    # Create dataframe with question, answer, Llm answer and scores
    df_results = pd.DataFrame(columns=['question', 'answer', 'LLM answer', 'F1', 'Exact Match'])

    for item in tqdm(questions_to_ask):
        question = item.get('Question')
        answer = normalize_answer(item.get('Answer'))

        # RAG context using search
        agent_answer = executor.invoke({"input": "Question: " + question})
        llm_answer = normalize_answer(agent_answer['output'])
        f1_score = token_f1_score(llm_answer, answer)
        e_match_score = 1 if llm_answer == answer else 0
        
        row = pd.Series([question, answer, llm_answer, f1_score, e_match_score], index=df_results.columns)
        df_results = pd.concat([df_results, pd.DataFrame([row])], ignore_index=True)

    return df_results

In [31]:
def run_agent(tool):
    prompt = PromptTemplate.from_template(LANGCHAIN_CHAT_SYSTEM_HEADER)
    llm = ChatGroq(temperature=0,
                model_name=LLM_MODEL_NAME,
                api_key=load_groq_key())
    tools = [tool]
    agent = create_react_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
    df = evaluate(agent_executor)
    return df

def print_results(df, results_file):
    display(df)
    df.to_csv(results_file)
    print()
    print("Metrics:")
    print(f"F1 score: Avg: {df['F1'].mean():.2f}. Std: {df['F1'].std():.2f}")
    print(f"Exact Match score: Avg: {df['Exact Match'].mean():.2f}. Std: {df['Exact Match'].std():.2f}")

### Searching with Tool returning top 5 hits with Rerank

In [32]:
df = run_agent(get_top5_rerank)

  0%|          | 0/50 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top5_rerank
Action Input: What is Zeus know for in Greek mythology?['Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion, who rules as king of the gods of Mount Olympus.His name is cognate with the first element of his Roman equivalent Jupiter.His mythologies and powers are similar, though not identical, to those of Indo-European deities such as Jupiter, Perkūnas, Perun, Indra and Thor.\n\n', 'Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion', "Greek mythology is the body of myths originally told by the ancient Greeks and a genre of Ancient Greek folklore.These stories concern the origin and the nature of the world, the lives and activities of deities, heroes, and mythological creatures, and the origins and significance of the ancient Greeks' own cult and ritual practices.Modern

  2%|▏         | 1/50 [00:02<02:08,  2.62s/it]

Do I need to use a tool? No
Final Answer: King of gods in Greek mythology

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top5_rerank
Action Input: How long had the First World War been over when Messe was named aide-de-camp?['he became aide-de-camp to King Victor Emmanuel III, holding this post from 1923 to 1927', 'World War I (often abbreviated as WWI or WW1), also known as the First World War, the Great War, the Seminal Catastrophe, and initially in North America as the European War, was a global war originating in Europe that lasted from 28 July 1914 to 11 November 1918.Contemporaneously described as "the war to end all wars", it led to the mobilisation of more than 70\xa0million military personnel, including 60 million Europeans, making it one of the largest wars in history.It is also one of the deadliest conflicts', 'The 1936 Summer Olympics (German: Olympische Sommerspiele 1936), officially known as the Games of the XI

  4%|▍         | 2/50 [00:04<01:56,  2.42s/it]

...

Do I need to use a tool? No
Final Answer: No, he was not alive.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top5_rerank
Action Input: Which of the battles Sherman fought during a winter campaign had higher Union casualty rate?['Sherman returned to serve under General Ulysses S. Grant in the winter of 1862 during the battles of forts Henry and Donelson.', "In the Battle of Lützen (German: Schlacht von Großgörschen, 2 May 1813), Napoleon I of France halted the advances of the Sixth Coalition after the French invasion of Russia and the massive French losses in the campaign.The Russian commander, Prince Peter Wittgenstein, attempting to forestall Napoleon's capture of Leipzig, attacked the isolated French right wing near Lützen, Germany.After a day of heavy fighting, the combined Prussian and Russian force retreated; due to French losses and a shortage of Fren", 'The Battle of Kulm was a battle near the town Kulm (Chlumec) an

 98%|█████████▊| 49/50 [18:38<00:27, 27.09s/it]

Do I need to use a tool? No
Final Answer: Fort Donelson had higher Union casualty rate.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top5_rerank
Action Input: Which country did some of the tribes escaped to after their deportation from northern Mexico?['Those who escaped deportation were dispersed across northern Mexico and some even settled across the international border in southern Arizona', 'In the Battle of Bautzen (20–21 May 1813) a combined Russian–Prussian army, that was massively outnumbered, was pushed back by Napoleon I of France but escaped destruction, some sources claiming that Michel Ney failed to block their retreat.The Prussians under Count Gebhard von Blücher and Russians under Prince Peter Wittgenstein, retreating after their defeat at Lützen were attacked by French forces under Napoleon.\n\nPrelude.', "In the Battle of Lützen (German: Schlacht von Großgörschen, 2 May 1813), Napoleon I of France halted t

100%|██████████| 50/50 [19:06<00:00, 22.93s/it]

Do I need to use a tool? No
Final Answer: United States

> Finished chain.


In [33]:
print_results(df, "results/react_top5_rerank.csv")

,question,answer,LLM answer,F1,Exact Match
0,What is Zeus know for in Greek mythology?,sky and thunder god,king of gods in greek mythology,0,0
1,How long had the First World War been over whe...,5,5 years,0.666667,0
2,How old was Messe when the First World War sta...,30,31,0,0
3,How long had Angela Scoular been acting profes...,2,14 years,0,0
4,What is the capacity of the stadium where Brun...,26688,mcg has 100024 capacity,0,0
5,Which stadium where Brunt played can hold more...,white hart lane,white hart lane,1.0,1
6,In which country was Wilhelm Müller born?,germany,germany,1.0,1
7,Which battle Wilhelm Müller fought in while in...,kulm,battle of kulm,0.5,0
8,How much time had passed between Wilhelm's ret...,9,9 years,0.666667,0
9,Which of Wilhelm's direct male descendants liv...,friedrich max müller,wilhelm max müller 1919,0.571429,0



Metrics:
F1 score: Avg: 0.44. Std: 0.43
Exact Match score: Avg: 0.30. Std: 0.46


In [34]:
# Add contexts for RAGAS
df['contexts'] = contexts

In [35]:
df

,question,answer,LLM answer,F1,Exact Match,contexts
0,What is Zeus know for in Greek mythology?,sky and thunder god,king of gods in greek mythology,0,0,"[Zeus (British English , North American Englis..."
1,How long had the First World War been over whe...,5,5 years,0.666667,0,[he became aide-de-camp to King Victor Emmanue...
2,How old was Messe when the First World War sta...,30,31,0,0,[World War I (often abbreviated as WWI or WW1)...
3,How long had Angela Scoular been acting profes...,2,14 years,0,0,[Angela Scoular appeared as Ruby Bartlett in O...
4,What is the capacity of the stadium where Brun...,26688,mcg has 100024 capacity,0,0,[Brunt returned to first-team action after eig...
5,Which stadium where Brunt played can hold more...,white hart lane,white hart lane,1.0,1,[White Hart Lane was a football stadium in Tot...
6,In which country was Wilhelm Müller born?,germany,germany,1.0,1,[Wilhelm Max Müller (15 May 1862 – July 1919) ...
7,Which battle Wilhelm Müller fought in while in...,kulm,battle of kulm,0.5,0,[In the Battle of Bautzen (20–21 May 1813) a c...
8,How much time had passed between Wilhelm's ret...,9,9 years,0.666667,0,[In 1814 he returned to his studies at Berlin....
9,Which of Wilhelm's direct male descendants liv...,friedrich max müller,wilhelm max müller 1919,0.571429,0,"[his grandson Wilhelm Max Müller, Wilhelm Max ..."


## Evaluation using RAGAS

In [37]:
# Data necessary for RAGAS evaluation
questions = df['question']
responses = df['LLM answer']
contexts = df['contexts']
ground_truths = df['answer']

In [39]:
# Code adapted from https://arize.com/blog/ragas-how-to-evaluate-rag-pipeline-phoenix
from datasets import Dataset

def generate_ragas_dataset(questions, responses, contexts, ground_truths):

    dataset_dict = {
        "question": questions, # From the dataset
        "answer": responses, # From Llama3
        "contexts": contexts,
        "ground_truth": ground_truths,
    }
    ds = Dataset.from_dict(dataset_dict)
    return ds

ragas_dataset = generate_ragas_dataset(questions, responses, contexts, ground_truths)

In [41]:
from ragas import evaluate
evaluation_result = evaluate(
    dataset=ragas_dataset,
    metrics=[answer_relevancy, faithfulness, context_recall, context_precision],
)
df_ragas = pd.DataFrame(evaluation_result.scores)

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)